# Purpose

This notebook demonstrates the model experimentation and finalization. It covers EDA, outlier treatment, transformation, training, model evaluation and comparison across models.

## Imports

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# impute missing values
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from scripts import CombinedAttributesAdder

In [61]:
import warnings

warnings.filterwarnings('ignore', message="pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.", 
                        category=FutureWarning)
warnings.filterwarnings('ignore', message="pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.",
                        category=FutureWarning)

In [62]:
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, string_cleaning,
    get_package_path, display_as_tabs, save_pipeline, load_pipeline, initialize_environment,
    load_dataset, save_dataset, DEFAULT_ARTIFACTS_PATH
)

import ta_lib.eda.api as eda
from xgboost import XGBRegressor
from ta_lib.regression.api import SKLStatsmodelOLS
from ta_lib.regression.api import RegressionComparison, RegressionReport
import ta_lib.reports.api as reports
from ta_lib.data_processing.api import Outlier

initialize_environment(debug=False, hide_warnings=True)

In [63]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# Initialization

In [64]:
artifacts_folder = DEFAULT_ARTIFACTS_PATH

In [65]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

# 3 Feature Engineering

The focus here is the `Pipeline` and not the model. Though the model would inform the pipeline that is needed to train the model, our focus is to set it up in such a way that it can be saved/loaded, tweaked for different model choices and so on.

## 3.1 Read the Train and Test Data

In [66]:
train_X = load_dataset(context, 'train/housing/features')
train_y = load_dataset(context, 'train/housing/target')
print(train_X.shape, train_y.shape)

test_X = load_dataset(context, 'test/housing/features')
test_y = load_dataset(context, 'test/housing/target')
print(test_X.shape, test_y.shape)

(16512, 9) (16512, 1)
(4128, 9) (4128, 1)


In [67]:
train_X

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
16507,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,<1H OCEAN
16508,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,INLAND
16509,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,<1H OCEAN
16510,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,<1H OCEAN


## 3.2 Feature Engineering Pipelines


#### General Steps in the Feature Transformation are as follows
 - Outlier Treatment
 - Encoding of Categorical Columns
 - Missing Values Imputation

In [68]:
cat_columns = ["ocean_proximity"]
num_columns = train_X.columns.drop('ocean_proximity')

#### Encoding


Some sample pipelines showcasing how to create column specific pipelines and integrating them overall is presented below

- Commonly target encoding is done for categorical variables with too many levels.
- We also group sparse levels. For fewer levels one hot encoding/label encoding is preferred.
- If there is one dominant level, we can use binary encoding.
- This will go into production code

In [69]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])



features_transformer = ColumnTransformer([
    ("cat", OneHotEncoder(), cat_columns),
    ("num", num_pipeline, num_columns),    
])


In [12]:
# Fit and transform the features
train_X = features_transformer.fit_transform(train_X)

# Get transformed column names after transformation
transformed_columns = features_transformer.named_transformers_['cat'].get_feature_names_out(input_features=cat_columns).tolist()
transformed_columns += num_columns.tolist() + ["rooms_per_household" ,"population_per_household","bedrooms_per_room"] # Convert to list and add numerical columns
transformed_columns



['ocean_proximity_<1H OCEAN',
 'ocean_proximity_INLAND',
 'ocean_proximity_ISLAND',
 'ocean_proximity_NEAR BAY',
 'ocean_proximity_NEAR OCEAN',
 'longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'population_per_household',
 'bedrooms_per_room']

In [13]:
# num_columns_to_exclude = len(features_transformer.transformers_[1][2])
# additional_columns = features_transformer.transformers_[1][2]
train_X = get_dataframe(
    train_X, 
    transformed_columns,
)

In [14]:
# Custom Transformations like these can be utilised
def _custom_data_transform(df, cols2keep=None):
    """Transformation to drop some columns in the data
    
    Parameters
    ----------
        df - pd.DataFrame
        cols2keep - columns to keep in the dataframe
    """
    cols2keep = cols2keep or []
    if len(cols2keep):
        return (df
                .select_columns(cols2keep))
    else:
        return df

# 4 Modelling

## 4.1 Modelling - Linear Regression

### 4.1.1 Model Training

In [15]:
lin_reg = Pipeline([
    ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':train_X.columns.tolist()})),
    ('Linear Regression', SKLStatsmodelOLS())
])
train_y=train_y.values.ravel()
lin_reg.fit(train_X, train_y)

Pipeline(steps=[('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x7fce0e19a310>,
                                     kw_args={'cols2keep': ['ocean_proximity_<1H '
                                                            'OCEAN',
                                                            'ocean_proximity_INLAND',
                                                            'ocean_proximity_ISLAND',
                                                            'ocean_proximity_NEAR '
                                                            'BAY',
                                                            'ocean_proximity_NEAR '
                                                            'OCEAN',
                                                            'longitude',
                                                            'latitude',
                                                            'housing_median_age',
                                                            'total_rooms',
                                                            'total_bedrooms',
                                                            'population',
                                                            'households',
                                                            'median_income',
                                                            'rooms_per_household',
                                                            'population_per_household',
                                                            'bedrooms_per_room']})),
                ('Linear Regression', SKLStatsmodelOLS())])

In [16]:
lin_reg["Linear Regression"].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.648
Method:                 Least Squares   F-statistic:                     1899.
Date:                Sun, 07 Apr 2024   Prob (F-statistic):               0.00
Time:                        21:36:41   Log-Likelihood:            -2.0731e+05
No. Observations:               16512   AIC:                         4.147e+05
Df Residuals:                   16495   BIC:                         4.148e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
intercept                  -2.396e+15   1.28e+16     -0.187      0.852   -2.76e+16    2.28e+16
ocean_proximity_<1H OCEAN   2.396e+15   1.28e+16      0.187      0.852   -2.28e+16    2.76e+16
ocean_proximity_INLAND      2.396e+15   1.28e+16      0.187      0.852   -2.28e+16    2.76e+16
ocean_proximity_ISLAND      2.396e+15   1.28e+16      0.187      0.852   -2.28e+16    2.76e+16
ocean_proximity_NEAR BAY    2.396e+15   1.28e+16      0.187      0.852   -2.28e+16    2.76e+16
ocean_proximity_NEAR OCEAN  2.396e+15   1.28e+16      0.187      0.852   -2.28e+16    2.76e+16
longitude                  -5.565e+04   2292.875    -24.271      0.000   -6.01e+04   -5.12e+04
latitude                   -5.671e+04   2415.398    -23.478      0.000   -6.14e+04    -5.2e+04
housing_median_age          1.374e+04    616.514     22.279      0.000    1.25e+04    1.49e+04
total_rooms                -1941.7686   2190.725     -0.886      0.375   -6235.825    2352.288
total_bedrooms              7343.9712   3199.101      2.296      0.022    1073.387    1.36e+04
population                 -4.571e+04   1374.670    -33.251      0.000   -4.84e+04    -4.3e+04
households                  4.545e+04   3145.000     14.452      0.000    3.93e+04    5.16e+04
median_income               7.471e+04    755.318     98.917      0.000    7.32e+04    7.62e+04
rooms_per_household         6604.0710    675.794      9.772      0.000    5279.443    7928.699
population_per_household    1042.7789    553.141      1.885      0.059     -41.438    2126.996
bedrooms_per_room           9248.0752    791.432     11.685      0.000    7696.784    1.08e+04
==============================================================================
Omnibus:                     4238.857   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20514.484
Skew:                           1.164   Prob(JB):                         0.00
Kurtosis:                       7.939   Cond. No.                     1.16e+14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.76e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 4.1.2 Model Metrics

In [17]:
housing_predictions = lin_reg.predict(train_X)
lin_mse = mean_squared_error(train_y, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68627.9534852309

In [18]:
lin_mae = mean_absolute_error(train_y, housing_predictions)
lin_mae

49451.62376000016

In [19]:
lin_scores = cross_val_score(lin_reg, train_X, train_y,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [71770.87401059 64093.54227344 67769.17136888 68628.96334013
 66836.90306843 72534.24799488 73996.10494749 68805.47079146
 66446.77274915 70129.05758301]
Mean: 69101.11081274619
Standard deviation: 2885.5499903285986


In [20]:
test_X = get_dataframe(
    features_transformer.transform(test_X), 
    transformed_columns
)

In [21]:
reg_linear_report = RegressionReport(model=lin_reg, x_train=train_X, y_train=train_y, x_test= test_X, y_test= test_y, refit=True)
reg_linear_report.get_report(include_shap=False, file_path='regression_linear_model_report')

## 4.2 Modelling - Decision Tree Regression

### 4.2.1 Model Training

In [22]:
tree_reg = Pipeline([
    ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':train_X.columns.tolist()})),
    ('Decision Tree', DecisionTreeRegressor(random_state=context.random_seed))
])
tree_reg.fit(train_X, train_y)

Pipeline(steps=[('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x7fce0e19a310>,
                                     kw_args={'cols2keep': ['ocean_proximity_<1H '
                                                            'OCEAN',
                                                            'ocean_proximity_INLAND',
                                                            'ocean_proximity_ISLAND',
                                                            'ocean_proximity_NEAR '
                                                            'BAY',
                                                            'ocean_proximity_NEAR '
                                                            'OCEAN',
                                                            'longitude',
                                                            'latitude',
                                                            'housing_median_age',
                                                            'total_rooms',
                                                            'total_bedrooms',
                                                            'population',
                                                            'households',
                                                            'median_income',
                                                            'rooms_per_household',
                                                            'population_per_household',
                                                            'bedrooms_per_room']})),
                ('Decision Tree', DecisionTreeRegressor(random_state=42))])

### 4.2.2 Model Metrics

In [23]:
housing_predictions = tree_reg.predict(train_X)
tree_mse = mean_squared_error(train_y, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [24]:
scores = cross_val_score(tree_reg, train_X, train_y,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

display_scores(tree_rmse_scores)


Scores: [73355.81009447 70061.73564688 68576.29117891 71318.05418586
 69221.00498221 78757.81328565 70504.63112173 74116.40460477
 69340.76365034 70833.53893086]
Mean: 71608.60476816757
Standard deviation: 2911.843733100056


In [25]:
reg_linear_report = RegressionReport(model=tree_reg, x_train=train_X, y_train=train_y, x_test= test_X, y_test= test_y, refit=True)
reg_linear_report.get_report(include_shap=False, file_path='decision_tree_model_report')

divide by zero encountered in true_divide
invalid value encountered in multiply
invalid value encountered in true_divide


## 4.3 Modelling - Random Forest Regression

### 4.3.1 Model Training

In [26]:
forest_reg = Pipeline([
    ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':train_X.columns.tolist()})),
    ('Decision Tree', RandomForestRegressor(n_estimators=100, random_state=context.random_seed))
])
forest_reg.fit(train_X, train_y)

Pipeline(steps=[('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x7fce0e19a310>,
                                     kw_args={'cols2keep': ['ocean_proximity_<1H '
                                                            'OCEAN',
                                                            'ocean_proximity_INLAND',
                                                            'ocean_proximity_ISLAND',
                                                            'ocean_proximity_NEAR '
                                                            'BAY',
                                                            'ocean_proximity_NEAR '
                                                            'OCEAN',
                                                            'longitude',
                                                            'latitude',
                                                            'housing_median_age',
                                                            'total_rooms',
                                                            'total_bedrooms',
                                                            'population',
                                                            'households',
                                                            'median_income',
                                                            'rooms_per_household',
                                                            'population_per_household',
                                                            'bedrooms_per_room']})),
                ('Decision Tree', RandomForestRegressor(random_state=42))])

### 4.3.2 Model Metrics

In [27]:
housing_predictions = forest_reg.predict(train_X)
forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18654.932100817703

In [28]:
forest_scores = cross_val_score(forest_reg, train_X, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [51416.20200373 48690.12407842 47116.1588672  51866.99691804
 47554.78605315 51891.27986946 52656.3719011  50057.54266081
 48665.06951673 54085.88378281]
Mean: 50400.04156514521
Standard deviation: 2213.9423430863103


### 4.3.3 Model Best Esitmator

In [29]:
parameters = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
est = RandomForestRegressor(random_state=context.random_seed)
grid_search = GridSearchCV(est, parameters, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(train_X, train_y)

print(grid_search.best_score_)
print(grid_search.best_params_)

-2490104051.6981955
{'max_features': 8, 'n_estimators': 30}


In [30]:
forest_reg= Pipeline([
    ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':train_X.columns.tolist()})),
    ('Random Forest', grid_search.best_estimator_)
])
forest_reg.fit(train_X, train_y)

Pipeline(steps=[('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x7fce0e19a310>,
                                     kw_args={'cols2keep': ['ocean_proximity_<1H '
                                                            'OCEAN',
                                                            'ocean_proximity_INLAND',
                                                            'ocean_proximity_ISLAND',
                                                            'ocean_proximity_NEAR '
                                                            'BAY',
                                                            'ocean_proximity_NEAR '
                                                            'OCEAN',
                                                            'longitude',
                                                            'latitude',
                                                            'housing_median_age',
                                                            'total_rooms',
                                                            'total_bedrooms',
                                                            'population',
                                                            'households',
                                                            'median_income',
                                                            'rooms_per_household',
                                                            'population_per_household',
                                                            'bedrooms_per_room']})),
                ('Random Forest',
                 RandomForestRegressor(max_features=8, n_estimators=30,
                                       random_state=42))])

In [31]:
reg_linear_report = RegressionReport(model=forest_reg, x_train=train_X, y_train=train_y, x_test= test_X, y_test= test_y, refit=True)
reg_linear_report.get_report(include_shap=False, file_path='random_forest_model_report')

## 4.4 SVR

### 4.4.1 Model Training

In [32]:
svm_reg = Pipeline([
    ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2keep':train_X.columns.tolist()})),
    ('SVR', SVR(kernel="linear"))
])
svm_reg.fit(train_X, train_y)

Pipeline(steps=[('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x7fce0e19a310>,
                                     kw_args={'cols2keep': ['ocean_proximity_<1H '
                                                            'OCEAN',
                                                            'ocean_proximity_INLAND',
                                                            'ocean_proximity_ISLAND',
                                                            'ocean_proximity_NEAR '
                                                            'BAY',
                                                            'ocean_proximity_NEAR '
                                                            'OCEAN',
                                                            'longitude',
                                                            'latitude',
                                                            'housing_median_age',
                                                            'total_rooms',
                                                            'total_bedrooms',
                                                            'population',
                                                            'households',
                                                            'median_income',
                                                            'rooms_per_household',
                                                            'population_per_household',
                                                            'bedrooms_per_room']})),
                ('SVR', SVR(kernel='linear'))])

In [33]:
housing_predictions = svm_reg.predict(train_X)
svm_mse = mean_squared_error(train_y, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

111095.06635291968

In [34]:
reg_linear_report = RegressionReport(model=svm_reg, x_train=train_X, y_train=train_y, x_test= test_X, y_test= test_y, refit=True)
reg_linear_report.get_report(include_shap=False, file_path='svr_model_report')

# 5. Model Comparison

In [35]:
model_pipelines = [lin_reg, tree_reg, forest_reg,  svm_reg]
model_comparison_report = RegressionComparison(models=model_pipelines,x=train_X, y=train_y, refit=True)
metrics = model_comparison_report.get_report(include_shap=False)

ValueError: Shape of passed values is (16512, 1), indices imply (16512, 16)

In [ ]:
model_comparison_report.performance_metrics